In [ ]:
# 각 시점마다 prediction 예측 후, n차원 형태로 미래 prediction 값을 제공해보자

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/kubig_financial/finrl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## install required packages
!pip install gym
!pip install stockstats
!pip install gymnasium
!pip install stable_baselines3
!pip install alpaca_trade_api
!pip install exchange_calendars
!pip install pyfolio
!pip install matplotlib
!pip install swig
!pip install wrds
!pip install pyportfolioopt
!pip install --upgrade ipykernel
## install finrl library
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 15.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 5.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import itertools

In [ ]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [ ]:
#model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
TRAIN_START_DATE = '2009-01-01' #실제 시작 시점은 2010년 0101이나, prediction용 1년치 추가함.
PREDICTION_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-05-05'

tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

In [ ]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = tickers).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (32481, 8)


In [ ]:
#종가 기준 Datasetting

data_for_pred = pd.DataFrame(index = range(len(df[df.tic==tickers[0]])), columns = tickers)
data_for_pred['date'] = df.date.unique()
for tic in tickers:
    data_for_pred[tic] = df[df.tic==tic].close.values

data_for_pred.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  getpass.getpass = self._save_getpass


,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,date
0,17.207544,31.549156,7.878144,18.167875,12.912431,16.372383,17.672304,21.070581,18.663012,2009-01-02
1,17.244053,32.291485,7.747468,18.025709,12.872055,16.440004,17.922132,20.969475,18.580608,2009-01-05
2,17.630985,32.643776,7.871922,18.422293,13.170840,16.196554,17.666357,20.627235,18.893724,2009-01-06
3,17.222155,31.259775,7.461218,17.599201,12.750925,16.034246,17.523600,20.440567,18.143904,2009-01-07
4,17.463068,31.662388,7.430101,17.771299,12.839752,15.824609,17.434380,20.557238,18.251024,2009-01-08


In [ ]:
prediction_start_index = data_for_pred[data_for_pred['date'] >= PREDICTION_START_DATE].index[0]
test_start_index = data_for_pred[data_for_pred['date'] >= TRADE_START_DATE].index[0]

print(prediction_start_index)
print(test_start_index)

252
3209


In [ ]:
idx_date = data_for_pred['date']
def idx_date_mapper(idx_list):
    date_list = []
    for idx in idx_list:
        date_list.append(idx_date[idx])
    return date_list

In [ ]:
data_for_pred = data_for_pred.drop('date',axis=1)
data_for_pred.head()

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,17.207544,31.549156,7.878151,18.167877,12.912434,16.372387,17.672312,21.070585,18.663015
1,17.244053,32.291492,7.747468,18.025703,12.872054,16.440001,17.922134,20.969473,18.580612
2,17.630980,32.643776,7.871928,18.422297,13.170841,16.196548,17.666363,20.627237,18.893717
3,17.222145,31.259773,7.461215,17.599190,12.750926,16.034248,17.523600,20.440561,18.143900
4,17.463064,31.662397,7.430101,17.771301,12.839752,15.824614,17.434378,20.557236,18.251020


In [ ]:
# Hyperparameters
WINDOW_SIZE = prediction_start_index
FORCAST_SIZE = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


train_dataset = data_for_pred[:test_start_index]
test_dataset = data_for_pred[test_start_index-WINDOW_SIZE:]

print(len(train_dataset),len(test_dataset))

3209 652


In [ ]:
class moving_avg(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = torch.nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

class series_decomp(torch.nn.Module):
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        residual = x - moving_mean
        return moving_mean, residual

class LTSF_DLinear(torch.nn.Module):
    def __init__(self, window_size, forcast_size, kernel_size, individual, feature_size):
        super(LTSF_DLinear, self).__init__()
        self.window_size = window_size
        self.forcast_size = forcast_size
        self.decompsition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear_Seasonal = torch.nn.ModuleList()
            self.Linear_Trend = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Trend[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
                self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Seasonal[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
        else:
            self.Linear_Trend = torch.nn.Linear(self.window_size, self.forcast_size)
            self.Linear_Trend.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
            self.Linear_Seasonal = torch.nn.Linear(self.window_size,  self.forcast_size)
            self.Linear_Seasonal.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

    def forward(self, x):
        trend_init, seasonal_init = self.decompsition(x)
        trend_init, seasonal_init = trend_init.permute(0,2,1), seasonal_init.permute(0,2,1)
        if self.individual:
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forcast_size], dtype=trend_init.dtype).to(trend_init.device)
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forcast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
            for idx in range(self.channels):
                trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :])
                seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :])
        else:
            trend_output = self.Linear_Trend(trend_init)
            seasonal_output = self.Linear_Seasonal(seasonal_init)
        x = seasonal_output + trend_output
        return x.permute(0,2,1)

In [ ]:
class Windowed_Dataset(Dataset):
    def __init__(self, data, window_size, forcast_size):
        self.data = data
        self.window_size = window_size
        self.forcast_size = forcast_size

    def __len__(self):
        return len(self.data) - self.window_size - self.forcast_size

    def __getitem__(self, idx):
        return torch.from_numpy(self.data[idx:idx+self.window_size].values).type(torch.FloatTensor), torch.from_numpy(self.data[idx+self.window_size:idx+self.window_size+self.forcast_size].values).type(torch.FloatTensor)


In [ ]:
WD_train = Windowed_Dataset(train_dataset, WINDOW_SIZE, FORCAST_SIZE)
WD_test = Windowed_Dataset(test_dataset, WINDOW_SIZE, FORCAST_SIZE)
WD_all = Windowed_Dataset(data_for_pred, WINDOW_SIZE, FORCAST_SIZE)

train_loader = DataLoader(WD_train, batch_size=32, shuffle=False)
test_loader = DataLoader(WD_test, batch_size=32, shuffle=False)
all_loader = DataLoader(WD_all, batch_size=32, shuffle=False)

print(len(WD_train), len(WD_test), len(WD_all))

2927 370 3327


In [ ]:
model = LTSF_DLinear(WINDOW_SIZE, FORCAST_SIZE, 25, True, len(tickers))
model = model.to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, criterion, optimizer, device, epochs, tolerance=0.0001):
    model.train()
    train_loss = 0
    for epoch in tqdm(range(epochs)):
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        if epoch % 10 == 0:
            print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, loss.item()))
        if train_loss < tolerance:
            print('Training Loss: {:.6f}'.format(loss.item()))
            break
        train_loss = 0

    return model

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    predictions = []
    for X, y in tqdm(test_loader):
        X = X.to(device)
        y = y.to(device)
        with torch.no_grad():
            output = model(X)
            loss = criterion(output, y)
            test_loss += loss.item()
            predictions.append(output)

    print('Test Loss: {:.6f}'.format(loss.item()))
    return predictions


In [ ]:
model = train(model, train_loader, criterion, optimizer, device, 200, 1)

  0%|          | 1/200 [00:02<08:44,  2.64s/it]

Epoch: 1 	Training Loss: 112.843628


  6%|▌         | 11/200 [00:13<03:03,  1.03it/s]

Epoch: 11 	Training Loss: 100.334557


 10%|█         | 21/200 [00:22<02:39,  1.13it/s]

...

 90%|█████████ | 181/200 [02:51<00:15,  1.20it/s]

Epoch: 181 	Training Loss: 136.992844


 96%|█████████▌| 191/200 [03:01<00:07,  1.17it/s]

Epoch: 191 	Training Loss: 128.598846


100%|██████████| 200/200 [03:09<00:00,  1.05it/s]


In [ ]:
PATH = '/content/drive/MyDrive/kubig_financial/finrl/result'
torch.save(model.state_dict(), PATH)

In [ ]:
model = LTSF_DLinear(WINDOW_SIZE, FORCAST_SIZE, 25, True, len(tickers))
model = model.to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
test_predictions = test(model, test_loader, criterion, device)

100%|██████████| 12/12 [00:00<00:00, 159.29it/s]

Test Loss: 826.522583


In [ ]:
test_predictions = torch.cat(test_predictions, dim=0)
test_predictions.shape

torch.Size([370, 30, 9])

In [ ]:
plt.figure(figsize=(12,8))
plt.title('LTSF-DLinear')
for i in range(9,30,10):
    plt.plot(test_predictions[:,i,0].cpu().detach().numpy(), label='Predictions_{}'.format(i+1))
plt.plot(test_dataset[tickers[0]].values[WINDOW_SIZE:-FORCAST_SIZE], label='True')
plt.legend()
plt.show()

#### Data 뽑기


In [ ]:
all_predictions = test(model, all_loader, criterion, device)
all_predictions = torch.cat(all_predictions, dim=0)
all_predictions.shape

100%|██████████| 104/104 [00:00<00:00, 186.02it/s]

Test Loss: 972.907776


torch.Size([3327, 30, 9])

In [ ]:
prediction_5 = pd.DataFrame(all_predictions[:,4,:], columns=tickers, index = (data_for_pred.index[WINDOW_SIZE:-FORCAST_SIZE]))
prediction_10 = pd.DataFrame(all_predictions[:,9,:], columns=tickers, index = (data_for_pred.index[WINDOW_SIZE:-FORCAST_SIZE]))
prediction_30 = pd.DataFrame(all_predictions[:,29,:], columns=tickers, index = (data_for_pred.index[WINDOW_SIZE:-FORCAST_SIZE]))

In [ ]:
prediction_5[tickers[0]].plot(figsize=(12,8), label='Predictions_5')
data_for_pred[tickers[0]].plot(label='True')
plt.legend()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  getpass.getpass = self._save_getpass
/usr/local/lib/python3.10/dist-packages/pandas/plotting/_matplotlib/core.py:792: MatplotlibDeprecationWarning: The legendHandles attribute was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use legend_handles instead.
  handles = leg.legendHandles
/usr/local/lib/python3.10/dist-packages/pandas/plotting/_matplotlib/core.py:792: MatplotlibDeprecationWarning: The legendHandles attribute was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use legend_handles instead.
  handles = leg.legendHandles


In [ ]:
prediction_5.to_csv('/content/drive/MyDrive/kubig_financial/dataset/prediction_5.csv')
prediction_10.to_csv('/content/drive/MyDrive/kubig_financial/dataset/prediction_10.csv')
prediction_30.to_csv('/content/drive/MyDrive/kubig_financial/dataset/prediction_30.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  getpass.getpass = self._save_getpass
